In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 

import matplotlib.pyplot as plt
import jax.numpy as jnp
import numpyro
import pandas as pd
import arviz as az

numpyro.set_host_device_count(4)

from epimodel import run_model, default_model, arma_model, latent_nn_model, EpidemiologicalParameters, preprocess_data
from epimodel.models.model_predict_utils import *
from epimodel.models.model_build_utils import *

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Importing plotly failed. Interactive plots will not work.


In [2]:
# dates for data processing
start_date="2020-08-01"
end_date="2020-12-10"

# Gets data
data = preprocess_data('../data/all_merged_data_2021-01-22.csv', start_date=start_date, end_date=end_date)
all_data = preprocess_data('../data/all_merged_data_2021-01-22.csv')

data.featurize()
all_data.featurize()


Processing data from 2020-08-01 00:00:00 to 2020-12-10 00:00:00
Processing data from 2020-08-01 00:00:00 to 2021-01-22 00:00:00
Note: under drop_outdoor gathering aggregation, the gatherings_aggregation_type is disregarded
Note: under drop_outdoor gathering aggregation, the gatherings_aggregation_type is disregarded


In [15]:
skip_crps=False
verbose=False

look_ahead = 23

cases_list = []

mse_cases_list = []
crps_cases_list = []

sum_mse_cases_list = []
sum_crps_cases_list = []


gradient_best_fit_cases = []

floor_cap_list_cases = []

cap_case_list = [10000, 12000, 24000, 36000, 48000, 60000]

for floor_case in [0]:
    for cap_case in cap_case_list:

        floor_cap = [floor_case, cap_case, 0, 100]
        floor_cap_list_cases.append(floor_cap)

        cases_total_prophet, deaths_total_prophet = prophet_predictor(data, look_ahead, start_date, end_date, floor_cap, nS=40)

        cases_mse_prophet, cases_crps_prophet = get_prediction_metrics(cases_total_prophet, all_data.new_cases.data, data.nDs, skip_crps=skip_crps, verbose=verbose)

        mse_cases_list.append(cases_mse_prophet)
        crps_cases_list.append(cases_crps_prophet)

        sum_mse_cases_list.append(jnp.sum(cases_mse_prophet[-23:]))
        sum_crps_cases_list.append(jnp.sum(cases_crps_prophet[-23:]))

        

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3

In [19]:
prediction_date = pd.to_datetime(end_date)
labels = ['cap=10000', 'cap=12000', 'cap=24000', 'cap=36000', 'cap=48000', 'cap=60000']


plot_graph(tuple(mse_cases_list), 'cases_mse_prophet.png', prediction_date,label=labels,region=None,title='Cases MSE')
plot_graph(tuple(crps_cases_list), 'cases_crps_prophet.png',prediction_date,label=labels,region=None,title='Cases CRPS')

<Figure size 432x288 with 0 Axes>

In [17]:
sum_mse_cases_list

[DeviceArray(9236468., dtype=float32),
 DeviceArray(2129149.2, dtype=float32),
 DeviceArray(2282673.2, dtype=float32),
 DeviceArray(2467921., dtype=float32),
 DeviceArray(2450318.2, dtype=float32),
 DeviceArray(2658042., dtype=float32)]

In [18]:
sum_crps_cases_list

[DeviceArray(4423.9067, dtype=float32),
 DeviceArray(3618.7107, dtype=float32),
 DeviceArray(3851.647, dtype=float32),
 DeviceArray(3964.2969, dtype=float32),
 DeviceArray(3953.9507, dtype=float32),
 DeviceArray(4034.6162, dtype=float32)]

In [20]:
floor_cap_list_cases

[[0, 10000, 0, 100],
 [0, 12000, 0, 100],
 [0, 24000, 0, 100],
 [0, 36000, 0, 100],
 [0, 48000, 0, 100],
 [0, 60000, 0, 100]]

In [42]:
skip_crps=False
verbose=False

look_ahead = 23

deaths_list = []

mse_deaths_list = []
crps_deaths_list = []

sum_crps_deaths_list = []
sum_mse_deaths_list = []

floor_cap_list_death = []

for floor_death in [0]: #1
    for cap_death in [300,450,800, 1600, 3200, 4800, 6400, 8000]:

        floor_cap = [0, 10000, floor_death, cap_death]
        floor_cap_list_death.append(floor_cap)

        cases_total_prophet, deaths_total_prophet = prophet_predictor(data, look_ahead, start_date, end_date, floor_cap, nS=40)

        deaths_mse_prophet, deaths_crps_prophet = get_prediction_metrics(deaths_total_prophet, all_data.new_deaths.data, data.nDs, skip_crps=skip_crps, verbose=verbose)

        mse_deaths_list.append(deaths_mse_prophet)
        crps_deaths_list.append(deaths_crps_prophet)

        sum_mse_deaths_list.append(jnp.sum(deaths_mse_prophet[-23:]))
        sum_crps_deaths_list.append(jnp.sum(deaths_crps_prophet[-23:]))


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/auto/users/ms21tal/miniforge3/envs/theo/lib/python3

In [43]:
prediction_date = pd.to_datetime(end_date)
labels = ['cap=300', 'cap=450', 'cap=800', 'cap=1600', 'cap=3200', 'cap=4800','cap=6400','cap=8000' ]

plot_graph(tuple(mse_deaths_list), 'deaths_mse_prophet.png', prediction_date,label=labels,region=None,title='Deaths MSE')
plot_graph(tuple(crps_deaths_list), 'deaths_crps_prophet.png',prediction_date,label=labels,region=None,title='Deaths MSE')


<Figure size 432x288 with 0 Axes>

In [44]:
sum_mse_deaths_list

[DeviceArray(10138.331, dtype=float32),
 DeviceArray(11620.121, dtype=float32),
 DeviceArray(9007.196, dtype=float32),
 DeviceArray(9814.464, dtype=float32),
 DeviceArray(8849.236, dtype=float32),
 DeviceArray(8087.0317, dtype=float32),
 DeviceArray(8355.406, dtype=float32),
 DeviceArray(8133.91, dtype=float32)]

In [46]:
sum_crps_deaths_list

[DeviceArray(163.00311, dtype=float32),
 DeviceArray(168.83871, dtype=float32),
 DeviceArray(159.7188, dtype=float32),
 DeviceArray(162.1248, dtype=float32),
 DeviceArray(152.54343, dtype=float32),
 DeviceArray(149.70628, dtype=float32),
 DeviceArray(149.2093, dtype=float32),
 DeviceArray(150.78568, dtype=float32)]

In [ ]:
floor_cap_list_death